In [1]:
import sys
import biogeme.biogeme_logging as blog
import biogeme.exceptions as excep
import biogeme.biogeme as bio
import biogeme.distributions as dist
import biogeme.results as res
from biogeme import models
from biogeme.expressions import (
    Beta,
    RandomVariable,
    exp,
    log,
    Integrate,
)

from read_or_estimate import read_or_estimate

from optima import (
    read_data,
    male,
    age,
    haveChildren,
    highEducation,
       SocioProfCat,
    WaitingTimePT,
    Choice,
    TimePT_scaled,
    TimeCar_scaled,
    MarginalCostPT_scaled,
    CostCarCHF_scaled,
    distance_km_scaled,
    PurpHWH,
    PurpOther,
)

logger = blog.get_screen_logger(level=blog.INFO)
logger.info('Example m02_sequential_estimation1.py')

Example m02_sequential_estimation1.py 


In [2]:
MODELNAME = 'm01_latent_variable'
try:
    struct_results = res.bioResults(pickle_file=f'{MODELNAME}.pickle')
except excep.BiogemeError:
    print(
        f'Run first the script {MODELNAME}.py in order to generate the '
        f'file {MODELNAME}.pickle, and move it to the directory saved_results'
    )
    sys.exit()
struct_betas = struct_results.get_beta_values()

In [3]:
coef_intercept = struct_betas['coef_intercept']
coef_age_50_less = struct_betas['coef_age_50_less']
coef_male = struct_betas['coef_male']
coef_haveChildren = struct_betas['coef_haveChildren']
coef_highEducation = struct_betas['coef_highEducation']
coef_employees = struct_betas['coef_employees']


In [4]:
omega = RandomVariable('omega')
density = dist.normalpdf(omega)
sigma_s = Beta('sigma_s', 1, None, None, 0)

In [5]:
ACTIVELIFE = (
    coef_intercept
        + coef_highEducation * highEducation
       + coef_employees * (SocioProfCat == 6)
    + coef_age_50_less * (age <= 50)
    + coef_male * male
    + coef_haveChildren * haveChildren
    + sigma_s * omega
)

In [6]:
ASC_CAR = Beta('ASC_CAR', 0.94, None, None, 0)
ASC_PT = Beta('ASC_PT', 0, None, None, 1)
ASC_SM = Beta('ASC_SM', 0.35, None, None, 0)
BETA_COST_HWH = Beta('BETA_COST_HWH', -2.3, None, None, 0)
BETA_COST_OTHER = Beta('BETA_COST_OTHER', -1.9, None, None, 0)
BETA_DIST = Beta('BETA_DIST', -1.3, None, None, 0)
BETA_TIME_CAR_REF = Beta('BETA_TIME_CAR_REF', -6.1, None, 0, 0)
BETA_TIME_PT_REF = Beta('BETA_TIME_PT_REF', 0, None, 0, 0)
BETA_WAITING_TIME = Beta('BETA_WAITING_TIME', -0.075, None, None, 0)

In [7]:
BETA_TIME_PT_AL = Beta('BETA_TIME_PT_AL', 1.5, None, None, 0)
BETA_TIME_PT = BETA_TIME_PT_REF + BETA_TIME_PT_AL * ACTIVELIFE
BETA_TIME_CAR_AL = Beta('BETA_TIME_CAR_AL', -0.11, None, None, 0)
BETA_TIME_CAR = BETA_TIME_CAR_REF + BETA_TIME_CAR_AL * ACTIVELIFE

In [8]:
V0 = (
    ASC_PT
    + BETA_TIME_PT * TimePT_scaled
    + BETA_WAITING_TIME * WaitingTimePT
    + BETA_COST_HWH * MarginalCostPT_scaled * PurpHWH
    + BETA_COST_OTHER * MarginalCostPT_scaled * PurpOther
)

V1 = (
    ASC_CAR
    + BETA_TIME_CAR * TimeCar_scaled
    + BETA_COST_HWH * CostCarCHF_scaled * PurpHWH
    + BETA_COST_OTHER * CostCarCHF_scaled * PurpOther
)

V2 = ASC_SM + BETA_DIST * distance_km_scaled

In [9]:
V = {0: V0, 1: V1, 2: V2}

In [10]:
condprob = models.logit(V, None, Choice)

In [11]:
loglike = log(Integrate(condprob * density, 'omega'))

In [12]:
database = read_data()

In [13]:
the_biogeme = bio.BIOGEME(database, loglike)
the_biogeme.modelName = 'm02_sequential_estimation1'
the_biogeme.maxiter = 1000

Biogeme parameters read from biogeme.toml. 


In [14]:
results = read_or_estimate(the_biogeme=the_biogeme, directory='saved_results')

As the model is rather complex, we cancel the calculation of second derivatives. If you want to control the parameters, change the name of the algorithm in the TOML file from "automatic" to "simple_bounds" 
*** Initial values of the parameters are obtained from the file __m02_sequential_estimation1.iter 
Parameter values restored from __m02_sequential_estimation1.iter 
As the model is rather complex, we cancel the calculation of second derivatives. If you want to control the parameters, change the name of the algorithm in the TOML file from "automatic" to "simple_bounds" 
Optimization algorithm: hybrid Newton/BFGS with simple bounds [simple_bounds] 
** Optimization: BFGS with trust region for simple bounds 
Iter.         ASC_CAR          ASC_SM   BETA_COST_HWH BETA_COST_OTHER       BETA_DIST BETA_TIME_CAR_A BETA_TIME_CAR_R BETA_TIME_PT_AL BETA_TIME_PT_RE BETA_WAITING_TI         sigma_s     Function    Relgrad   Radius      Rho      
    0            0.96            0.37            -2.2

In [15]:
print(results.short_summary())

Results for model m02_sequential_estimation1
Nbr of parameters:		11
Sample size:			1906
Excluded data:			0
Final log likelihood:		-1151.957
Akaike Information Criterion:	2325.915
Bayesian Information Criterion:	2386.995



In [16]:
printprint(results.short_summary())(f'Final log likelihood: {results.data.logLike:.3f}')
print(f'Output file: {results.data.htmlFileName}')

Final log likelihood: -1151.957
Output file: m02_sequential_estimation1~00.html


In [17]:
results.get_estimated_parameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_CAR,0.564037,0.133543,4.223649,2.403785e-05
ASC_SM,0.373352,0.178836,2.087674,3.682722e-02
BETA_COST_HWH,-2.154634,0.182496,-11.806458,0.000000e+00
BETA_COST_OTHER,-1.352969,0.143999,-9.395667,0.000000e+00
BETA_DIST,-2.310992,0.050866,-45.433147,0.000000e+00
BETA_TIME_CAR_AL,-1.614503,0.226472,-7.128936,1.011413e-12
BETA_TIME_CAR_REF,-17.478038,1.203559,-14.521965,0.000000e+00
BETA_TIME_PT_AL,-0.481458,0.063949,-7.528752,5.129230e-14
BETA_TIME_PT_REF,-5.436746,0.595814,-9.124901,0.000000e+00
BETA_WAITING_TIME,-0.033013,0.009550,-3.456741,5.467510e-04


In [18]:
print(results.short_summary())

Results for model m02_sequential_estimation1
Nbr of parameters:		11
Sample size:			1906
Excluded data:			0
Final log likelihood:		-1151.957
Akaike Information Criterion:	2325.915
Bayesian Information Criterion:	2386.995

